# Store

> Module for constructing AppletStore.

In [ ]:
#| default_exp store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import yaml
import json
import os

from llmcam.oas_to_requests import toolbox_schema, generate_request
from typing import Optional

## Dynamic API installation

In [ ]:
#| export
def load_oas(
    oas_url: str = "https://tie.digitraffic.fi/swagger/openapi.json",  # OpenAPI Specification URL
    destination: str = "api/road_digitraffic.json",  # Destination file
    overwrite: bool = False  # Overwrite existing file
) -> dict:  # OpenAPI Specification
    """Load OpenAPI Specification from URL or file."""
    # Create destination directory if it does not exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)

    # Download OpenAPI Specification if it does not exist or overwrite is True
    if not os.path.exists(destination) or overwrite:
        r = requests.get(oas_url)
        with open(destination, "w") as f:
            f.write(r.text)

    # Load OpenAPI Specification
    with open(destination, "r") as f:
        if destination.endswith(".json"):
            return json.load(f)
        elif destination.endswith(".yaml") or destination.endswith(".yml"):
            return yaml.load(f)
        else:
            raise ValueError("Invalid file format")

In [ ]:
show_doc(load_oas)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L18){target="_blank" style="float:right; font-size:smaller"}

### load_oas

>      load_oas (oas_url:str='https://tie.digitraffic.fi/swagger/openapi.json',
>                destination:str='api/road_digitraffic.json',
>                overwrite:bool=False)

*Load OpenAPI Specification from URL or file.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| oas_url | str | https://tie.digitraffic.fi/swagger/openapi.json | OpenAPI Specification URL |
| destination | str | api/road_digitraffic.json | Destination file |
| overwrite | bool | False | Overwrite existing file |
| **Returns** | **dict** |  | **OpenAPI Specification** |

Usage to download three DigiTraffic endpoints

In [ ]:
road_digitraffic = load_oas(
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    destination="api/road_digitraffic.json",
    overwrite=False
)
train_digitraffic = load_oas(
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    destination="api/train_digitraffic.json",
    overwrite=False
)
marine_digitraffic = load_oas(
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    destination="api/marine_digitraffic.json",
    overwrite=False
)

Test functions with GPT:

In [ ]:
from llmcam.fn_to_fc import complete, form_msg, form_msgs, print_msgs
from llmcam.oas_to_requests import generate_request

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get me information about the trains departing on 2017-11-09 with train number 1."),
])
complete(messages, toolbox_schema("https://rata.digitraffic.fi", train_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
Here is the information about train number 1 on the departure date 2017-11-09:  - **Operator**: VR -
**Train Type**: InterCity (IC) - **Train Category**: Long-distance - **Running Currently**: No -
**Cancelled**: No - **Timetable Type**: Regular - **Timetable Acceptance Date**: 2017-07-21
**Selected Stops**: - **Helsinki (HKI)**: Departure at 05:30, 3 minutes late. - **Pasila (PSL)**:
Arrival at 05:35, 2 minutes late; Departure at 05:37, 3 minutes late. - **Tikkurila (TKL)**: Arrival
at 05:45, 2 minutes late; Departure at 05:47, 3 minutes late. - **Lahdessa (LH)**: Arrival at 06:22,
7 minutes late; Departure at 06:24, 6 minutes late. - **Lahti (LR)**: Arrival at 07:30, 5 minutes
late; Departure at 07:31, 5 minutes late. - **Joensuu (JNS)**: Final Arrival at 09:52, 8 minutes
late.  The train ha

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get me information about the trains departing on 2017-11-09 with train number 1.")
])
complete(messages, toolbox_schema("https://rata.digitraffic.fi", train_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
Here is the information for Train Number 1 that departed on 2017-11-09:  - **Operator:** Finnish
Railways (VR) - **Train Type:** InterCity (IC) - **Train Category:** Long-distance - **Running
Currently:** No - **Cancelled:** No  ### Timetable Summary:  1. **Helsinki (HKI)**    -
**Departure:** Scheduled at 05:28, Actual 05:30:33 (Difference: 3 mins)  2. **Pasila (PSL)**    -
**Arrival:** Scheduled at 05:33, Actual 05:35:26 (Difference: 2 mins)    - **Departure:** Scheduled
at 05:34, Actual 05:37:13 (Difference: 3 mins)  3. **Tikkurila (TKL)**    - **Arrival:** Scheduled
at 05:43, Actual 05:45:29 (Difference: 2 mins)    - **Departure:** Scheduled at 05:44, Actual
05:47:20 (Difference: 3 mins)  4. **Lahti (LH)**    - **Arrival:** Scheduled at 06:16, Actual
06:22:39 (Difference: 7 mins)    - **De

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Are there any active nautical warnings?")
])
complete(messages, toolbox_schema("https://meri.digitraffic.fi", marine_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Are there any active nautical warnings?
>> Assistant:
Yes, there are several active nautical warnings. Here are a few examples:  1. **Gulf of Finland**
- **Type:** Local Warning    - **Content:** Leading light Skarvörarna Front (NR 11407) in position
59-58.85N 024-08.33E is out of order.    - **Location:** Inkoo Fairway  2. **Sea of Bothnia**    -
**Type:** Coastal Warning    - **Content:** Kaskinen approaches have 30 cubic meters of timber
drifting in position 62-09N 021-02E.    - **Date and Time:** 162217 UTC of November  3.
**Archipelago Sea**    - **Type:** Navigational Warnings for Yachtsmen    - **Content:** The fairway
is closed for traffic due to bridge construction from 20.11.2024 to 30.4.2025.    - **Location:**
Hessund Fairway  4. **Northern Baltic**    - **Type:** Coastal Warning    - **Content:** Military
exercise "Freezing Winds" ongoing. Vessels should navigate with cauti

In [ ]:
#| export
def add_api_tools(
    tools: list,  # List of existing tools
    service_name: str,  # Name of the API service
    base_url: str,  # Base URL of the API service
    oas_url: Optional[str] = None,  # OpenAPI Specification URL
    oas_destination: Optional[str] = None # OpenAPI Specification destination file
):
    """Add API tools to the toolbox."""
    # Load OpenAPI Specification
    if oas_url is None:
        oas_url = f"{base_url}/swagger/openapi.json"
    if oas_destination is None:
        oas_destination = f"api/{service_name}.json"
    oas = load_oas(oas_url, oas_destination, overwrite=True)

    # Create tool schema and append to toolbox
    schema = toolbox_schema(base_url, oas, service_name=service_name, fixup=generate_request)
    tools.extend(schema)

In [ ]:
show_doc(add_api_tools)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L43){target="_blank" style="float:right; font-size:smaller"}

### add_api_tools

>      add_api_tools (tools:list, service_name:str, base_url:str,
>                     oas_url:Optional[str]=None,
>                     oas_destination:Optional[str]=None)

*Add API tools to the toolbox.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| tools | list |  | List of existing tools |
| service_name | str |  | Name of the API service |
| base_url | str |  | Base URL of the API service |
| oas_url | Optional | None | OpenAPI Specification URL |
| oas_destination | Optional | None | OpenAPI Specification destination file |

Example usage:

In [ ]:
tools = []
add_api_tools(tools, "road_digitraffic", "https://tie.digitraffic.fi")
assert len(tools) > 0
tools[0]

{'type': 'function',
 'function': {'name': 'tmsStationsDatex2Xml',
  'description': 'The static information of TMS stations in Datex2 format (Traffic Measurement System / LAM)',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'object',
     'properties': {'state': {'type': 'string',
       'description': 'Road station state',
       'default': 'ACTIVE',
       'enum': ['ALL', 'REMOVED', 'ACTIVE']}},
     'required': []}},
   'required': []},
  'metadata': {'url': 'https://tie.digitraffic.fi/api/beta/tms-stations-datex2.xml',
   'method': 'get',
   'accepted_queries': ['state'],
   'service': 'road_digitraffic'},
  'fixup': 'llmcam.oas_to_requests.generate_request'}}

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get the weather camera information for the stations with ID C01503 and C01504."),
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the information for the weather camera stations with IDs C01503 and C01504:  ### Weather
Camera Station C01503 - **Name:** kt51_Inkoo - **Camera Type:** BOSCH - **Coordinates:** Longitude
23.99616, Latitude 60.05374 - **Nearest Weather Station ID:** 1013 - **Collection Status:**
GATHERING - **Data Updated Time:** 2024-11-19T15:25:43Z - **Collection Interval:** 600 seconds -
**Location Details:**    - **Municipality:** Inkoo (Municipality Code: 149)   - **Province:**
Uusimaa (Province Code: 1)   - **Road Number:** 51   - **Municipality:** Inkoo  **Presets:** 1.
**Inkooseen**    - **Direction:** INCREASING_DIRECTION    - **Resolution:** 1280x720    - **Image
URL:** [Inkooseen Image](https://weathercam.digitraffic.fi/C0150301.jpg) 2. **Hankoon**    -
**Direction:** DECREASING_DIRECTION    -

## Dynamic functions from installed libraries

It is possible to dynamically import a function in installed library and execute it with `importlib`.

In [ ]:
#| export
from importlib import import_module
from typing import Callable, Any
from llmcam.fn_to_fc import tool_schema

Utility to add functions from installed libraries to ToolBox.

In [ ]:
#| export
def add_function_tools(
    tools: list,  # List of existing tools
    service_name: str,  # Name of the service
    function_names: list[str],  # List of function names (with module prefix)
):
    """Add function tools to the toolbox."""
    # Import functions
    for function_name in function_names:
        # Get module prefix
        module_prefix = function_name.split(".")
        if len(module_prefix) == 1:
            module_prefix = "builtins"
        else:
            module_prefix = ".".join(module_prefix[:-1])

        # Get function name without module prefix
        func_name = function_name.split(".")[-1]

        # Import function
        if module_prefix == "builtins":
            func: Callable = getattr(__builtins__, func_name, None)
        else:
            func: Callable = getattr(import_module(module_prefix), func_name, None)

        # Raise error if function not found
        if func is None:
            raise ValueError(f"Function not found: {function_name}")
        
        # Create tool schema and append to toolbox
        tools.append(tool_schema(func=func, service_name=service_name))

In [ ]:
show_doc(add_function_tools)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L68){target="_blank" style="float:right; font-size:smaller"}

### add_function_tools

>      add_function_tools (tools:list, service_name:str,
>                          function_names:list[str])

*Add function tools to the toolbox.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| tools | list | List of existing tools |
| service_name | str | Name of the service |
| function_names | list | List of function names (with module prefix) |

Example usage:

In [ ]:
tools = []
add_function_tools(
    tools, 
    "youtube_live", 
    [
        "llmcam.fn_to_fc.capture_youtube_live_frame_and_save", 
        "llmcam.fn_to_fc.ask_gpt4v_about_image_file"
    ])
assert len(tools) == 2
tools

[{'type': 'function',
  'function': {'name': 'capture_youtube_live_frame_and_save',
   'description': 'Capture a jpeg file from YouTube Live and save in data directory',
   'parameters': {'type': 'object',
    'properties': {'link': {'anyOf': [{'type': 'string',
        'description': 'YouTube Live link'},
       {'type': 'null',
        'description': 'A default value will be automatically used.'}]},
     'place': {'anyOf': [{'type': 'string',
        'description': 'Location of live image'},
       {'type': 'null',
        'description': 'A default value will be automatically used.'}]}},
    'required': []},
   'metadata': {'module': 'llmcam.fn_to_fc', 'service': 'youtube_live'}}},
 {'type': 'function',
  'function': {'name': 'ask_gpt4v_about_image_file',
   'description': 'Tell all about quantitative information from a given image file',
   'parameters': {'type': 'object',
    'properties': {'path': {'type': 'string',
      'description': 'Path to the image file'}},
    'required': 

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    ("user", "Hi, can you capture and extract information from a YouTube Live? Use the default link.")
])
complete(messages, tools=tools)
print_msgs(messages)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
cap_2024.11.20_03:56:10_unclear.jpg
>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hi, can you capture and extract information from a YouTube Live? Use the default link.
>> Assistant:
I've captured an image from the default YouTube Live link and extracted the following information:
- **Timestamp:** 2024-11-20T03:48:23 - **Location:** Valkosaari - **Image Dimensions:** 1280 x 720
pixels - **Buildings:** There's 1 building with a height range of 2-3 stories. - **Vehicles:** No
vehicles are visible. - **Water Bodies:** A harbor is visible, with 1 boat present. - **Street
Lights:** There are 3 street lights. - **

## ToolBox management as FC

While the base functions for adding functions and APIs as tools work for appending new functions to an existing Python list, these functions cannot be easily transformed into tools to be used for function calling because:

- Changes must be made into the top-level `tools` and continue to be applied in all subsequent conversations.
- `tools` cannot be safely interpreted as a part of message as in some cases new tool changes may exceed token limit.

One solution to this is:

1. For each `tools` list, create corresponding handlers which manages this `tools` instance.
2. These handlers are registered into a `holder` dictionary that can be accessed globally by a unique session ID for each user session.
3. `holder_name` and `session_id` are saved as metadata for the handler functions.
4. A special fix-up function is used for handlers which accesses and executes the handlers when called via `holder_name` (accessed globally) and `session_id`.

In [ ]:
#| export

# Additional functions
def remove_tools(
    tools: list,  # List of existing tools
    service_name: str  # Name of the service
):
    """Remove tools from the toolbox."""
    tools[:] = [tool for tool in tools if ("service" not in tool["function"]["metadata"] or \
                                       tool["function"]["metadata"]["service"] != service_name)]

In [ ]:
#| export
import uuid
import importlib
from copy import deepcopy

In [ ]:
#| export
def execute_handler(
    function_name: str,  # Name of the function to execute
    module: str,  # Module of the function
    session_id: str,  # Session ID
    toolboxes_name: str = "session_tools",  # Name of the holder for toolboxes per session in globals spacce
    **kwargs  # Keyword arguments
):
    """Execute the handler function by retrieving function with session ."""
    # Get the holder
    holder = globals().get(toolboxes_name, None)
    if holder is None:
        raise ValueError(f"Holder not found: {toolboxes_name}")

    # Get the toolbox for the session
    tools = holder.get(session_id, None)
    if tools is None:
        raise ValueError(f"Session tools not found: {session_id}")
    
    # Get the function
    module = importlib.import_module(module)
    function = getattr(module, function_name, None)
    if function is None:
        raise ValueError(f"Function not found: {function_name}")
    
    if "service" in kwargs:
        del kwargs["service"]
    
    # Execute the function
    function(tools, **kwargs)

In [ ]:
#| export
def handler_schema(
    function: Callable,  # Handler function
    session_id: str,  # Session ID
    toolboxes_name: str = "session_tools",  # Name of the holder for toolboxes per session in globals spacce
    service_name: str = "toolbox_handler",  # Name of the service
    fixup: Optional[Callable] = None  # Function to fixup function execution
):
    """Create a schema for handlers."""
    schema = tool_schema(function, )
    schema["function"]["metadata"]["session_id"] = session_id
    schema["function"]["metadata"]["toolboxes_name"] = toolboxes_name
    schema["function"]["metadata"]["service"] = service_name
    if fixup: schema["function"]['fixup'] = f"{fixup.__module__}.{fixup.__name__}"

    if "tools" in schema["function"]["parameters"]["properties"]:
        del schema["function"]["parameters"]["properties"]["tools"]
        schema["function"]["parameters"]["required"].remove("tools")
    
    return schema

In [ ]:
#| export
def initialize_handlers(
        functions: list[Callable],  # List of functions to initialize
        toolboxes_name: str = "session_tools",  # Name of the holder for toolboxes per session in globals spacce
        service_name: str = "toolbox_handler",  # Name of the service
        fixup: Optional[Callable] = None,  # Function to fixup function execution
    ):
    """Initialize handlers"""
    # Create the holder for toolboxes per session in globals space
    session_tools = globals().get(toolboxes_name, None)
    if session_tools is None:
        session_tools = {}
        globals()[toolboxes_name] = session_tools

    # Initialize the handler schema
    session_id = str(uuid.uuid4())
    tools = [ handler_schema(function, session_id, toolboxes_name, service_name, fixup) for function in functions ]

    # Add the tools to the holder
    session_tools[session_id] = tools

    return session_id, tools

Example usage:

In [ ]:
session_tools = {}
session_id, tools = initialize_handlers(
    [add_api_tools, add_function_tools, remove_tools], 
    toolboxes_name="session_tools",
    service_name="toolbox_handler",
    fixup=execute_handler
)
assert len(session_tools) == 1
assert len(session_tools[session_id]) == 3

session_tools[session_id][0]

{'type': 'function',
 'function': {'name': 'add_api_tools',
  'description': 'Add API tools to the toolbox.',
  'parameters': {'type': 'object',
   'properties': {'service_name': {'type': 'string',
     'description': 'Name of the API service'},
    'base_url': {'type': 'string',
     'description': 'Base URL of the API service'},
    'oas_url': {'anyOf': [{'type': 'string',
       'description': 'OpenAPI Specification URL'},
      {'type': 'null',
       'description': 'A default value will be automatically used.'}]},
    'oas_destination': {'anyOf': [{'type': 'string',
       'description': 'OpenAPI Specification destination file'},
      {'type': 'null',
       'description': 'A default value will be automatically used.'}]}},
   'required': ['service_name', 'base_url']},
  'metadata': {'module': '__main__',
   'service': 'toolbox_handler',
   'session_id': '370d2426-bfcc-4495-8f8b-e38283c9b209',
   'toolboxes_name': 'session_tools'},
  'fixup': '__main__.execute_handler'}}

In [ ]:
execute_handler(
    "add_api_tools",
    "__main__",
    session_id=session_id,
    service_name="road_digitraffic",
    base_url="https://tie.digitraffic.fi"
)
assert len(session_tools[session_id]) > 3
session_tools[session_id][4]

{'type': 'function',
 'function': {'name': 'tmsStationsDatex2Json',
  'description': 'The static information of TMS stations in Datex2 format (Traffic Measurement System / LAM)',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'object',
     'properties': {'state': {'type': 'string',
       'description': 'Road station state',
       'default': 'ACTIVE',
       'enum': ['ALL', 'REMOVED', 'ACTIVE']}},
     'required': []}},
   'required': []},
  'metadata': {'url': 'https://tie.digitraffic.fi/api/beta/tms-stations-datex2.json',
   'method': 'get',
   'accepted_queries': ['state'],
   'service': 'road_digitraffic'},
  'fixup': 'llmcam.oas_to_requests.generate_request'}}

Simulated GPT workflow:

In [ ]:
session_tools = {}
session_id, tools = initialize_handlers(
    [add_api_tools, add_function_tools, remove_tools], 
    toolboxes_name="session_tools",
    service_name="toolbox_handler",
    fixup=execute_handler
)

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    ("user", "Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.")
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and
the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.
>> Assistant:
The new API service 'road_digitraffic' has been successfully added with the base URL
'https://tie.digitraffic.fi' and the OpenAPI Specification URL
'https://tie.digitraffic.fi/swagger/openapi.json'.


In [ ]:
#| eval: false
len(session_tools[session_id])

64

In [ ]:
#| eval: false
messages.append(form_msg("user", "Get the weather camera information for the stations with ID C01503 and C01504."))
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and
the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.
>> Assistant:
The new API service 'road_digitraffic' has been successfully added with the base URL
'https://tie.digitraffic.fi' and the OpenAPI Specification URL
'https://tie.digitraffic.fi/swagger/openapi.json'.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with IDs C01503 and C01504:  ### Station
C01503: kt51_Inkoo - **Camera Type:** BOSCH - **Location Coordinates:** [23.99616, 60.05374] -
**Municipality:** Inkoo - **Province:** Uusimaa - **Collection Status:** Gathering - **Collection
Interval:** 600 seconds (10 minutes) - **Presets:**   - **Inkooseen:**     - Resolution: 1280x720
- Direction: 

In [ ]:
#| eval: false
messages.append(form_msg("user", "Remove the 'road_digitraffic' service."))
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Add a new API service called 'road_digitraffic'. Use the base URL 'https://tie.digitraffic.fi', and
the OpenAPI Specification URL 'https://tie.digitraffic.fi/swagger/openapi.json'.
>> Assistant:
The new API service 'road_digitraffic' has been successfully added with the base URL
'https://tie.digitraffic.fi' and the OpenAPI Specification URL
'https://tie.digitraffic.fi/swagger/openapi.json'.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with IDs C01503 and C01504:  ### Station
C01503: kt51_Inkoo - **Camera Type:** BOSCH - **Location Coordinates:** [23.99616, 60.05374] -
**Municipality:** Inkoo - **Province:** Uusimaa - **Collection Status:** Gathering - **Collection
Interval:** 600 seconds (10 minutes) - **Presets:**   - **Inkooseen:**     - Resolution: 1280x720
- Direction: 

In [ ]:
#| eval: false
# After removing the 'road_digitraffic' service, the tools should only contain the initial handlers
len(tools)

3

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()